# **Chapter 28: Mobile-Specific Scenarios**

---

## **28.1 Introduction to Mobile-Specific Scenarios**

Mobile applications operate in a dynamic environment where system states change rapidly and unpredictably. Unlike desktop applications that enjoy stable execution contexts, mobile apps must respond gracefully to hardware transformations, system resource constraints, and user context switches.

**Why Mobile Scenarios Are Unique:**

1. **Lifecycle Complexity:** Apps transition between active, inactive, background, and suspended states within seconds
2. **Hardware Intimacy:** Direct integration with sensors, biometric scanners, and cameras requires handling hardware unavailability and permission revocation
3. **Contextual Awareness:** Location changes, network handovers, and device orientation shifts happen continuously during app usage
4. **Resource Volatility:** Memory pressure can terminate background apps without warning; storage can fill up unexpectedly
5. **Security Boundaries:** Sandboxing and permission models restrict access dynamically based on user choices

This chapter provides comprehensive testing strategies for these complex scenarios, ensuring your application maintains data integrity, user experience consistency, and functional reliability regardless of environmental changes.

---

## **28.2 Orientation Changes and Multi-Window Modes**

Device orientation changes trigger configuration changes that can destroy and recreate activities (Android) or resize view controllers (iOS), potentially causing data loss or state corruption.

### **28.2.1 Screen Rotation Handling**

```python
class OrientationTesting:
    """
    Testing device orientation changes and configuration updates
    """
    
    def test_portrait_to_landscape_transition(self, driver):
        """
        Verify UI adapts and data persists during rotation
        """
        # Set portrait orientation
        driver.orientation = 'PORTRAIT'
        
        # Navigate to form screen
        driver.find_element(AppiumBy.ID, "edit_profile").click()
        
        # Populate form fields
        name_field = driver.find_element(AppiumBy.ID, "full_name")
        name_field.send_keys("John Doe")
        
        email_field = driver.find_element(AppiumBy.ID, "email")
        email_field.send_keys("john@example.com")
        
        # Scroll to save button (might be needed in portrait)
        driver.swipe(500, 1000, 500, 500, 800)
        
        # Capture current scroll position or UI state
        current_focus = driver.find_element(AppiumBy.ID, "email").is_displayed()
        
        # Rotate to landscape
        driver.orientation = 'LANDSCAPE'
        time.sleep(1.5)  # Wait for rotation animation and layout recalculation
        
        # Verify form data persisted across configuration change
        name_after_rotation = driver.find_element(AppiumBy.ID, "full_name").text
        email_after_rotation = driver.find_element(AppiumBy.ID, "email").text
        
        assert name_after_rotation == "John Doe", "Name lost during rotation"
        assert email_after_rotation == "john@example.com", "Email lost during rotation"
        
        # Verify landscape-specific layout elements
        try:
            side_panel = driver.find_element(AppiumBy.ID, "landscape_side_panel")
            assert side_panel.is_displayed(), "Landscape layout not applied"
        except NoSuchElementException:
            # Alternative: Verify element positions changed appropriately
            element_location = driver.find_element(AppiumBy.ID, "save_button").location
            screen_width = driver.get_window_size()['width']
            screen_height = driver.get_window_size()['height']
            
            # In landscape, width > height
            assert screen_width > screen_height, "Screen dimensions incorrect for landscape"
    
    def test_continuous_rotation_stress(self, driver):
        """
        Stress test rapid orientation changes
        """
        driver.orientation = 'PORTRAIT'
        
        # Start a long-running operation (e.g., image upload)
        driver.find_element(AppiumBy.ID, "upload_photo").click()
        driver.find_element(AppiumBy.ID, "select_image").click()
        
        # Rotate multiple times during operation
        for i in range(5):
            driver.orientation = 'LANDSCAPE' if i % 2 == 0 else 'PORTRAIT'
            time.sleep(0.5)  # Rapid switching
        
        # Verify app didn't crash
        assert driver.find_element(AppiumBy.ID, "app_root").is_displayed()
        
        # Verify upload continued or handled interruption gracefully
        status = driver.find_element(AppiumBy.ID, "upload_status")
        assert "Error" not in status.text or "Retry" in status.text, "Upload failed catastrophically due to rotation"
    
    def test_foldable_device_postures(self, driver):
        """
        Test foldable devices (Samsung Galaxy Fold, Pixel Fold, etc.)
        """
        # Check if device supports foldable features
        is_foldable = self.check_foldable_support(driver)
        
        if not is_foldable:
            pytest.skip("Device not foldable")
        
        # Test folded state (outer screen - typically phone-sized)
        self.set_fold_state(driver, "folded")
        time.sleep(1)
        
        compact_element = driver.find_element(AppiumBy.ID, "compact_layout_indicator")
        assert compact_element.is_displayed()
        
        screen_size_folded = driver.get_window_size()
        assert screen_size_folded['width'] < 1000, "Screen too large for folded state"
        
        # Unfold device
        self.set_fold_state(driver, "unfolded")
        time.sleep(1.5)  # Wait for layout recalculation
        
        # Verify expanded layout
        expanded_element = driver.find_element(AppiumBy.ID, "expanded_layout_indicator")
        assert expanded_element.is_displayed()
        
        # Check for multi-pane layout (tablet-style)
        try:
            detail_pane = driver.find_element(AppiumBy.ID, "detail_pane")
            master_pane = driver.find_element(AppiumBy.ID, "master_pane")
            assert detail_pane.is_displayed() and master_pane.is_displayed()
        except NoSuchElementException:
            # Single pane but scaled appropriately
            pass
        
        # Test half-folded (laptop/tabletop mode) if supported
        try:
            self.set_fold_state(driver, "half_open")
            time.sleep(1)
            # Verify media controls or specific half-folded UI
        except:
            pass  # Not all foldables support this via software
    
    def check_foldable_support(self, driver):
        """Check if testing on foldable device"""
        try:
            # Android foldable detection
            result = driver.execute_script('mobile: shell', {
                'command': 'getprop ro.hardware.fold_type'
            })
            return bool(result)
        except:
            return False
    
    def set_fold_state(self, driver, state):
        """
        Control fold state via emulator commands or test APIs
        """
        if state == "folded":
            driver.execute_script('mobile: fold', {'action': 'fold'})
        elif state == "unfolded":
            driver.execute_script('mobile: fold', {'action': 'unfold'})
```

### **28.2.2 Multi-Window and Picture-in-Picture Testing**

```python
def test_multi_window_mode(self, driver):
    """
    Test Android split-screen and iOS Split View behavior
    """
    if driver.capabilities['platformName'] == 'Android':
        # Enter split-screen mode
        driver.execute_script('mobile: shell', {
            'command': 'am start -n com.android.systemui/.recents.RecentsActivity'
        })
        time.sleep(1)
        
        # Select split-screen (device-specific UI)
        # This requires specific device interaction
        
        # Verify app handles resize
        size = driver.get_window_size()
        assert size['width'] < driver.execute_script('mobile: shell', {
            'command': 'wm size'
        }).split('x')[0]  # Width should be half screen
        
        # Test interaction while in split-screen
        driver.find_element(AppiumBy.ID, "content").click()
        assert driver.find_element(AppiumBy.ID, "detail_view").is_displayed()

def test_picture_in_picture(self, driver):
    """
    Test video PiP mode (Android 8.0+, iOS 14+)
    """
    # Start video playback
    driver.find_element(AppiumBy.ID, "play_video").click()
    time.sleep(2)  # Let video start
    
    # Trigger PiP mode
    if driver.capabilities['platformName'] == 'Android':
        # Press home button (should trigger PiP if enabled in manifest)
        driver.press_keycode(3)  # HOME key
    else:
        # iOS PiP activation
        driver.find_element(AppiumBy.ID, "pip_button").click()
    
    time.sleep(1)
    
    # Verify PiP window exists (small window overlay)
    # This requires checking window manager or specific element visibility
    
    # Return to full screen by tapping PiP window
    driver.find_element(AppiumBy.ID, "pip_window").click()
    
    # Verify resumed in full screen
    assert driver.find_element(AppiumBy.ID, "video_controls").is_displayed()
```

---

## **28.3 Background and Foreground Modes**

Mobile operating systems aggressively manage background apps to preserve battery life. Testing must verify graceful handling of backgrounding, suspension, and termination.

### **28.3.1 App Lifecycle State Transitions**

```python
class BackgroundForegroundTesting:
    """
    Testing app behavior across lifecycle states
    """
    
    def test_background_foreground_data_persistence(self, driver):
        """
        Verify data persists when app backgrounds and resumes
        """
        # Create in-progress content
        driver.find_element(AppiumBy.ID, "new_note").click()
        driver.find_element(AppiumBy.ID, "note_content").send_keys("Draft content here")
        
        # Background app for 5 seconds
        driver.background_app(5)
        
        # App automatically returns to foreground after 5s
        # Verify content persisted
        note_text = driver.find_element(AppiumBy.ID, "note_content").text
        assert "Draft content here" in note_text, "Data lost during backgrounding"
        
        # Background for extended period (simulating user switching apps)
        driver.background_app(60)  # 1 minute
        
        # On iOS, app might be suspended; on Android, might be in cached state
        # Verify app handles resume gracefully (might show splash briefly)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((AppiumBy.ID, "note_content"))
        )
        
        note_text_after = driver.find_element(AppiumBy.ID, "note_content").text
        assert "Draft content here" in note_text_after, "Data lost after extended background"
    
    def test_system_termination_recovery(self, driver):
        """
        Test behavior when OS kills app in background (low memory)
        """
        # iOS: Simulate memory pressure termination
        # Android: Force stop via ADB
        
        # Create unsaved work
        driver.find_element(AppiumBy.ID, "compose_email").click()
        driver.find_element(AppiumBy.ID, "recipient").send_keys("test@example.com")
        driver.find_element(AppiumBy.ID, "subject").send_keys("Important")
        driver.find_element(AppiumBy.ID, "body").send_keys("Draft email content")
        
        # Background app
        driver.background_app(-1)  # -1 keeps it in background indefinitely
        
        # Simulate system termination
        if driver.capabilities['platformName'] == 'Android':
            driver.execute_script('mobile: shell', {
                'command': 'am force-stop com.example.app'
            })
        else:
            # iOS: Terminate via XCUI or wait for memory pressure
            driver.terminate_app("com.example.app")
        
        # Relaunch app
        driver.activate_app("com.example.app")
        
        # Check if state restoration worked
        try:
            draft_indicator = driver.find_element(AppiumBy.ID, "draft_restored_banner")
            assert draft_indicator.is_displayed()
            
            # Verify content restored
            body_text = driver.find_element(AppiumBy.ID, "body").text
            assert "Draft email content" in body_text
        except NoSuchElementException:
            # If no auto-save implemented, should at least not crash
            pass  # Acceptable if app shows fresh start
    
    def test_doze_mode_behavior(self, driver):
        """
        Test Android Doze mode and App Standby (API 23+)
        """
        if driver.capabilities['platformName'] != 'Android':
            pytest.skip("Doze mode is Android-specific")
        
        # Schedule background work (e.g., sync)
        driver.find_element(AppiumBy.ID, "schedule_sync").click()
        
        # Simulate device entering Doze mode
        driver.execute_script('mobile: shell', {
            'command': 'dumpsys deviceidle force-idle'
        })
        
        # Wait for maintenance window or verify deferred
        time.sleep(10)
        
        # Exit Doze
        driver.execute_script('mobile: shell', {
            'command': 'dumpsys deviceidle unforce'
        })
        
        # Verify background work completed or scheduled appropriately
        sync_status = driver.find_element(AppiumBy.ID, "sync_status")
        assert "Scheduled" in sync_status.text or "Completed" in sync_status.text
    
    def test_foreground_service_limits(self, driver):
        """
        Test Android 8.0+ foreground service restrictions
        """
        # Start location tracking or music playback (foreground service)
        driver.find_element(AppiumBy.ID, "start_tracking").click()
        
        # Verify foreground service notification appears
        if driver.capabilities['platformName'] == 'Android':
            # Open notification shade
            driver.open_notifications()
            
            service_notification = driver.find_element(
                AppiumBy.XPATH, 
                "//android.widget.TextView[contains(@text, 'Tracking active')]"
            )
            assert service_notification.is_displayed()
            
            # Dismiss notifications and return to app
            driver.press_keycode(4)  # Back button
    
    def test_app_switcher_snapshot(self, driver):
        """
        Verify app switcher card shows correct content (security/privacy)
        """
        # Navigate to sensitive screen (credit card entry)
        driver.find_element(AppiumBy.ID, "payment_method").click()
        driver.find_element(AppiumBy.ID, "card_number").send_keys("4111111111111111")
        
        # Open app switcher
        if driver.capabilities['platformName'] == 'Android':
            driver.press_keycode(187)  # APP_SWITCH key
        else:
            # iOS: Double tap home or swipe up gesture
            driver.execute_script('mobile: pressButton', {'name': 'home'})
            time.sleep(0.5)
            driver.execute_script('mobile: pressButton', {'name': 'home'})
        
        time.sleep(1)
        
        # Take screenshot of app switcher card
        # Verify sensitive data is obscured or blank (FLAG_SECURE on Android, 
        # Application does not support backgrounding on iOS)
        
        # Return to app
        driver.activate_app("com.example.app")
        
        # Verify card data still there (field should persist)
        card_displayed = driver.find_element(AppiumBy.ID, "card_number").text
        assert "4111" in card_displayed or "••••" in card_displayed
```

---

## **28.4 Push Notifications**

Push notifications are critical for user engagement but require careful testing of delivery, display, and interaction across different app states.

### **28.4.1 Notification Delivery and Display**

```python
class PushNotificationTesting:
    """
    Comprehensive push notification testing
    """
    
    def test_foreground_notification_handling(self, driver):
        """
        Test notification received while app in foreground
        """
        # Ensure app is in foreground
        assert driver.find_element(AppiumBy.ID, "home_screen").is_displayed()
        
        # Send push notification via FCM (Firebase) or APNs
        self.send_test_push_notification(
            title="Test Alert",
            body="You have a new message",
            data={"screen": "messages", "id": "123"}
        )
        
        time.sleep(3)  # Wait for delivery
        
        # Foreground handling varies:
        # iOS: System notification banner or in-app handling
        # Android: Can be intercepted by app or shown as heads-up
        
        try:
            # Check if in-app alert shown (custom UI)
            in_app_alert = driver.find_element(AppiumBy.ID, "in_app_notification")
            assert in_app_alert.is_displayed()
            in_app_alert.click()
        except NoSuchElementException:
            # Check system notification
            driver.open_notifications()
            notif = driver.find_element(AppiumBy.XPATH, "//*[contains(@text, 'Test Alert')]")
            assert notif.is_displayed()
            notif.click()
        
        # Verify deep link handled
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((AppiumBy.ID, "message_screen"))
        )
    
    def test_background_notification_delivery(self, driver):
        """
        Test notification delivered while app in background
        """
        # Background app
        driver.background_app(-1)
        
        # Send notification
        self.send_test_push_notification(
            title="Background Test",
            body="Tap to open"
        )
        
        time.sleep(2)
        
        # Open notification shade
        driver.open_notifications()
        
        # Find and tap notification
        notif = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((AppiumBy.XPATH, "//*[contains(@text, 'Background Test')]"))
        )
        notif.click()
        
        # Verify app launched and handled payload
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((AppiumBy.ID, "app_root"))
        )
    
    def test_notification_channels_android(self, driver):
        """
        Test Android 8.0+ notification channels
        """
        if driver.capabilities['platformName'] != 'Android':
            pytest.skip("Notification channels are Android-specific")
        
        # Trigger notification from different channels
        channels = ["promotions", "messages", "system_alerts"]
        
        for channel in channels:
            self.send_test_push_notification(
                title=f"{channel} Test",
                body="Testing channel",
                android_channel_id=channel
            )
            time.sleep(2)
        
        # Open system notification settings
        driver.execute_script('mobile: shell', {
            'command': 'am start -a android.settings.APP_NOTIFICATION_SETTINGS -e android.provider.extra.APP_PACKAGE com.example.app'
        })
        
        # Verify channels listed
        for channel in channels:
            channel_elem = driver.find_element(AppiumBy.XPATH, f"//*[@text='{channel}']")
            assert channel_elem.is_displayed()
    
    def test_rich_notifications(self, driver):
        """
        Test notifications with images, actions, and input
        """
        # Send rich notification
        self.send_test_push_notification(
            title="Rich Notification",
            body="Check out this image",
            image_url="https://example.com/image.jpg",
            actions=["Reply", "Dismiss", "View"]
        )
        
        time.sleep(3)
        driver.open_notifications()
        
        # Verify image loaded (check for ImageView or specific layout)
        try:
            notif_image = driver.find_element(AppiumBy.ID, "android:id/big_picture")
            assert notif_image.is_displayed()
        except:
            pass  # Image might be loaded asynchronously
        
        # Test action button
        action_button = driver.find_element(AppiumBy.XPATH, "//*[@text='Reply']")
        action_button.click()
        
        # Verify inline reply or app opened to reply screen
        try:
            reply_input = driver.find_element(AppiumBy.ID, "inline_reply")
            reply_input.send_keys("Test reply")
            driver.find_element(AppiumBy.ID, "send_reply").click()
        except:
            # App opened instead
            assert driver.find_element(AppiumBy.ID, "reply_screen").is_displayed()
    
    def test_notification_permission_flow(self, driver):
        """
        Test notification permission request and handling
        """
        # Reset notification permissions
        if driver.capabilities['platformName'] == 'Android':
            driver.execute_script('mobile: shell', {
                'command': 'cmd notification allow_listener com.example/com.example.NotificationService'
            })
            # Actually reset: pm revoke com.example.app android.permission.POST_NOTIFICATIONS
        else:
            # iOS: Reset via settings or uninstall/reinstall
            driver.remove_app("com.example.app")
            driver.install_app("/path/to/app.ipa")
            driver.activate_app("com.example.app")
        
        # Trigger action that requests notification permission
        driver.find_element(AppiumBy.ID, "enable_notifications").click()
        
        # Handle permission dialog
        try:
            if driver.capabilities['platformName'] == 'Android':
                allow_btn = driver.find_element(AppiumBy.ID, "com.android.permissioncontroller:id/permission_allow_button")
            else:
                allow_btn = driver.find_element(AppiumBy.XPATH, "//XCUIElementTypeButton[@name='Allow']")
            
            allow_btn.click()
            
            # Verify permission granted
            driver.find_element(AppiumBy.ID, "notifications_enabled_indicator")
            
        except NoSuchElementException:
            # Permission already granted or denied
            pass
    
    def send_test_push_notification(self, title, body, **kwargs):
        """
        Helper to send FCM/APNs notification via test API
        """
        # Implementation using Firebase Admin SDK or APNs provider
        import requests
        
        payload = {
            "to": "/topics/test",
            "notification": {
                "title": title,
                "body": body
            },
            "data": kwargs.get('data', {})
        }
        
        # Send via your backend or FCM directly
        requests.post(
            "https://fcm.googleapis.com/fcm/send",
            json=payload,
            headers={"Authorization": "key=SERVER_KEY"}
        )
```

---

## **28.5 Location Services**

Location testing requires handling GPS accuracy, permission levels, background updates, and power optimization.

### **28.5.1 GPS and Location Accuracy**

```python
class LocationTesting:
    """
    Testing location-based features
    """
    
    def test_location_permission_levels(self, driver):
        """
        Test different permission levels (iOS: Never/While Using/Always, Android: Coarse/Fine/Background)
        """
        # Test "While Using App" only
        self.set_location_permission(driver, "while_using")
        
        driver.find_element(AppiumBy.ID, "get_location").click()
        
        # Should work in foreground
        location_text = driver.find_element(AppiumBy.ID, "current_location").text
        assert "Unknown" not in location_text
        
        # Background the app
        driver.background_app(5)
        
        # Try to get background location (should fail or show warning)
        if driver.capabilities['platformName'] == 'iOS':
            # iOS will pause updates
            pass
        else:
            # Android might allow limited updates depending on version
            pass
        
        # Now test "Always" permission
        self.set_location_permission(driver, "always")
        driver.find_element(AppiumBy.ID, "enable_background_tracking").click()
        
        # Should allow background tracking setup
        status = driver.find_element(AppiumBy.ID, "tracking_status")
        assert "Active" in status.text or "Background" in status.text
    
    def test_mock_location_handling(self, driver):
        """
        Test with simulated GPS coordinates
        """
        # Enable mock locations (developer options)
        if driver.capabilities['platformName'] == 'Android':
            # Set mock location via Appium or ADB
            driver.execute_script('mobile: shell', {
                'command': 'appium extended-geo-fix 37.7749 -122.4194 10'  # San Francisco
            })
            
            # Trigger location fetch
            driver.find_element(AppiumBy.ID, "refresh_location").click()
            time.sleep(2)
            
            # Verify app received mock location
            location_display = driver.find_element(AppiumBy.ID, "location_text")
            assert "San Francisco" in location_display.text or "37.7749" in location_display.text
        
        # iOS: Use GPX files or location simulation in Xcode Scheme
    
    def test_geofencing(self, driver):
        """
        Test geofence entry/exit events
        """
        # Create geofence
        driver.find_element(AppiumBy.ID, "set_geofence").click()
        driver.find_element(AppiumBy.ID, "radius").send_keys("100")
        driver.find_element(AppiumBy.ID, "save_geofence").click()
        
        # Simulate entering geofence
        if driver.capabilities['platformName'] == 'Android':
            # Mock location to inside geofence
            driver.execute_script('mobile: shell', {
                'command': 'am broadcast -a mock.location.ENTER --ei latitude 37.7749 --ei longitude -122.4194'
            })
        
        time.sleep(2)
        
        # Verify entry notification or UI update
        try:
            entry_alert = driver.find_element(AppiumBy.ID, "geofence_entry_alert")
            assert entry_alert.is_displayed()
        except:
            # Check notification shade for geofence notification
            driver.open_notifications()
            notif = driver.find_element(AppiumBy.XPATH, "//*[contains(@text, 'Entered')]")
            assert notif.is_displayed()
    
    def test_location_accuracy_modes(self, driver):
        """
        Test high accuracy vs battery saving modes
        """
        # Set high accuracy
        if driver.capabilities['platformName'] == 'Android':
            driver.execute_script('mobile: shell', {
                'command': 'settings put secure location_mode 3'  # High accuracy
            })
        
        driver.find_element(AppiumBy.ID, "get_precise_location").click()
        time.sleep(2)
        
        precise_location = driver.find_element(AppiumBy.ID, "accuracy_indicator").text
        
        # Set battery saving (network only)
        if driver.capabilities['platformName'] == 'Android':
            driver.execute_script('mobile: shell', {
                'command': 'settings put secure location_mode 2'  # Battery saving
            })
        
        driver.find_element(AppiumBy.ID, "get_precise_location").click()
        time.sleep(2)
        
        battery_location = driver.find_element(AppiumBy.ID, "accuracy_indicator").text
        
        # Verify app handles different accuracy levels appropriately
        assert "High" in precise_location or "Low" in battery_location
    
    def set_location_permission(self, driver, level):
        """Helper to set location permission level"""
        if driver.capabilities['platformName'] == 'Android':
            commands = {
                "denied": "pm revoke com.example.app android.permission.ACCESS_FINE_LOCATION",
                "while_using": "pm grant com.example.app android.permission.ACCESS_FINE_LOCATION",  # Foreground only
                "always": "pm grant com.example.app android.permission.ACCESS_BACKGROUND_LOCATION"
            }
            driver.execute_script('mobile: shell', {'command': commands.get(level)})
        else:
            # iOS: Use privacy settings or XCUI to change permissions
            pass
```

---

## **28.6 Biometric Authentication**

Touch ID and Face ID provide convenient security but require testing enrollment states, fallback mechanisms, and security boundaries.

### **28.6.1 Fingerprint and Face ID Testing**

```python
class BiometricTesting:
    """
    Testing Touch ID, Face ID, and fallback mechanisms
    """
    
    def test_successful_biometric_auth(self, driver):
        """
        Test successful fingerprint/face recognition
        """
        # Navigate to protected feature
        driver.find_element(AppiumBy.ID, "secure_vault").click()
        
        # Trigger biometric prompt
        driver.find_element(AppiumBy.ID, "unlock_with_biometric").click()
        
        # Wait for system biometric prompt
        time.sleep(1)
        
        # Simulate successful biometric (iOS)
        if driver.capabilities['platformName'] == 'iOS':
            driver.execute_script('mobile: sendBiometricMatch', {
                'type': 'faceId',  # or 'touchId'
                'match': True
            })
        else:
            # Android: Use BiometricPrompt test APIs or ADB
            driver.execute_script('mobile: sendBiometricMatch', {'match': True})
        
        # Verify access granted
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((AppiumBy.ID, "vault_contents"))
        )
        assert driver.find_element(AppiumBy.ID, "secret_files").is_displayed()
    
    def test_failed_biometric_fallback(self, driver):
        """
        Test failed biometric attempts fall back to passcode/password
        """
        driver.find_element(AppiumBy.ID, "secure_vault").click()
        driver.find_element(AppiumBy.ID, "unlock_with_biometric").click()
        
        time.sleep(1)
        
        # Simulate failed biometric (3 times to trigger fallback)
        for _ in range(3):
            if driver.capabilities['platformName'] == 'iOS':
                driver.execute_script('mobile: sendBiometricMatch', {
                    'type': 'faceId',
                    'match': False
                })
            time.sleep(0.5)
        
        # Verify fallback to passcode prompt appears
        try:
            passcode_field = driver.find_element(AppiumBy.ID, "passcode_field")
            assert passcode_field.is_displayed()
            
            # Enter passcode
            passcode_field.send_keys("123456")
            driver.find_element(AppiumBy.ID, "submit_passcode").click()
            
            # Verify access granted
            assert driver.find_element(AppiumBy.ID, "vault_contents").is_displayed()
        except NoSuchElementException:
            # Check for system passcode UI
            pass
    
    def test_biometric_enrollment_change(self, driver):
        """
        Test behavior when biometric enrollment changes (new fingerprint added)
        """
        # On iOS, adding new fingerprint invalidates Secure Enrollment
        # On Android, depends on implementation (CryptoObject)
        
        # Authenticate successfully first
        self.test_successful_biometric_auth(driver)
        driver.find_element(AppiumBy.ID, "lock_vault").click()
        
        # Simulate enrollment change (requires specific device state)
        # This is typically tested manually or with mocked security providers
        
        # Try to authenticate again
        driver.find_element(AppiumBy.ID, "unlock_with_biometric").click()
        
        # Should prompt for passcode due to invalidated biometrics
        # Or show error that biometrics changed
    
    def test_biometric_availability(self, driver):
        """
        Test graceful degradation when biometrics unavailable
        """
        # Check if device supports biometrics
        has_biometric = driver.execute_script('mobile: isBiometricEnrolled')
        
        if not has_biometric:
            # Should show passcode option only
            driver.find_element(AppiumBy.ID, "secure_vault").click()
            
            # Biometric button should be hidden or disabled
            try:
                bio_btn = driver.find_element(AppiumBy.ID, "unlock_with_biometric")
                assert not bio_btn.is_enabled() or not bio_btn.is_displayed()
            except NoSuchElementException:
                pass  # Correctly hidden
            
            # Passcode option should be available
            passcode_btn = driver.find_element(AppiumBy.ID, "unlock_with_passcode")
            assert passcode_btn.is_displayed()
```

---

## **28.7 Deep Linking**

Deep links allow external apps or web pages to open specific screens within your app. Testing must verify URL handling, state restoration, and attribution tracking.

### **28.7.1 Universal Links and App Links**

```python
class DeepLinkTesting:
    """
    Testing deep link handling
    """
    
    def test_universal_link_ios(self, driver):
        """
        Test iOS Universal Links (HTTPS URLs that open app)
        """
        if driver.capabilities['platformName'] != 'iOS':
            pytest.skip("Universal Links are iOS-specific")
        
        # Background the app
        driver.background_app(-1)
        
        # Open Safari and trigger universal link
        # Or use simulator command
        driver.execute_script('mobile: terminateApp', {'bundleId': 'com.apple.mobilesafari'})
        driver.execute_script('mobile: launchApp', {'bundleId': 'com.apple.mobilesafari'})
        
        # Navigate to universal link
        url_bar = driver.find_element(AppiumBy.XPATH, "//XCUIElementTypeTextField[@name='Address']")
        url_bar.send_keys("https://example.com/product/123")
        driver.find_element(AppiumBy.XPATH, "//XCUIElementTypeButton[@name='Go']").click()
        
        # Should open app instead of Safari (if universal link configured)
        time.sleep(2)
        
        # Verify app opened to correct screen
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((AppiumBy.ID, "product_detail_screen"))
        )
        
        product_id = driver.find_element(AppiumBy.ID, "product_id").text
        assert product_id == "123"
    
    def test_android_app_links(self, driver):
        """
        Test Android App Links (verified deep links)
        """
        if driver.capabilities['platformName'] != 'Android':
            pytest.skip("App Links are Android-specific")
        
        # Use ADB to send view intent
        driver.execute_script('mobile: shell', {
            'command': 'am start -W -a android.intent.action.VIEW -d "https://example.com/product/456" com.example.app'
        })
        
        # Verify app opened directly (no disambiguation dialog)
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((AppiumBy.ID, "product_detail_screen"))
        )
        
        product_id = driver.find_element(AppiumBy.ID, "product_id").text
        assert product_id == "456"
    
    def test_custom_url_scheme(self, driver):
        """
        Test custom scheme deep links (myapp://screen/data)
        """
        scheme = "myapp://profile/user123?source=email_campaign"
        
        if driver.capabilities['platformName'] == 'Android':
            driver.execute_script('mobile: shell', {
                'command': f'am start -W -a android.intent.action.VIEW -d "{scheme}" com.example.app'
            })
        else:
            driver.execute_script('mobile: terminateApp', {'bundleId': 'com.example.app'})
            driver.execute_script('mobile: shell', {
                'command': f'xcrun simctl openurl booted "{scheme}"'
            })
        
        # Verify navigation to profile screen
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((AppiumBy.ID, "profile_screen"))
        )
        
        user_id = driver.find_element(AppiumBy.ID, "user_id_display").text
        assert user_id == "user123"
        
        # Verify attribution captured
        source = driver.find_element(AppiumBy.ID, "attribution_source")
        assert "email_campaign" in source.text
    
    def test_deferred_deep_linking(self, driver):
        """
        Test deferred deep links (install from link, then open to specific screen)
        """
        # This requires integration with Branch, Adjust, or Firebase Dynamic Links
        # Typically tested by:
        # 1. Uninstalling app
        # 2. Clicking deferred deep link
        # 3. Installing app from store (or test build)
        # 4. Verifying first open goes to correct screen
        
        # Simulate first launch after install from deferred link
        driver.execute_script('mobile: shell', {
            'command': 'am start -a android.intent.action.MAIN -c android.intent.category.LAUNCHER com.example.app --ez deferred_link true --es deep_link_value "promo/50off"'
        })
        
        # Should show promo screen instead of home
        try:
            promo_screen = driver.find_element(AppiumBy.ID, "promo_screen")
            assert promo_screen.is_displayed()
            assert "50% Off" in driver.find_element(AppiumBy.ID, "promo_title").text
        except NoSuchElementException:
            # Fallback to normal home is acceptable if deferred link expired
            pass
    
    def test_deep_link_with_auth(self, driver):
        """
        Test deep links that require authentication
        """
        # Ensure logged out
        driver.reset()  # Fresh install
        
        # Trigger deep link to protected resource
        if driver.capabilities['platformName'] == 'Android':
            driver.execute_script('mobile: shell', {
                'command': 'am start -a android.intent.action.VIEW -d "myapp://orders/789" com.example.app'
            })
        
        # Should redirect to login first
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((AppiumBy.ID, "login_screen"))
        )
        
        # Login
        driver.find_element(AppiumBy.ID, "username").send_keys("testuser")
        driver.find_element(AppiumBy.ID, "password").send_keys("password")
        driver.find_element(AppiumBy.ID, "login_button").click()
        
        # Should automatically navigate to originally requested deep link
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((AppiumBy.ID, "order_detail_screen"))
        )
        
        order_id = driver.find_element(AppiumBy.ID, "order_number").text
        assert "789" in order_id
```

---

## **28.8 App Permissions**

Modern mobile OS use runtime permission models. Testing must cover permission requests, denials, "Don't ask again," and rationale dialogs.

### **28.8.1 Runtime Permission Flows**

```python
class PermissionTesting:
    """
    Comprehensive permission testing
    """
    
    def test_camera_permission_grant(self, driver):
        """
        Test granting camera permission
        """
        driver.find_element(AppiumBy.ID, "take_photo").click()
        
        # Permission dialog should appear
        if driver.capabilities['platformName'] == 'Android':
            allow_btn = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((AppiumBy.ID, "com.android.permissioncontroller:id/permission_allow_foreground_only_button"))
            )
            allow_btn.click()
        else:
            allow_btn = driver.find_element(AppiumBy.XPATH, "//XCUIElementTypeButton[@name='OK' or @name='Allow']")
            allow_btn.click()
        
        # Camera interface should appear
        assert driver.find_element(AppiumBy.ID, "camera_preview").is_displayed()
    
    def test_permission_denial_handling(self, driver):
        """
        Test graceful handling when permission denied
        """
        driver.find_element(AppiumBy.ID, "take_photo").click()
        
        # Deny permission
        if driver.capabilities['platformName'] == 'Android':
            deny_btn = driver.find_element(AppiumBy.ID, "com.android.permissioncontroller:id/permission_deny_button")
            deny_btn.click()
        else:
            deny_btn = driver.find_element(AppiumBy.XPATH, "//XCUIElementTypeButton[@name='Don't Allow']")
            deny_btn.click()
        
        # Should show explanation or fallback
        error_msg = driver.find_element(AppiumBy.ID, "permission_denied_message")
        assert error_msg.is_displayed()
        assert "Camera access is required" in error_msg.text
        
        # Alternative option should be available (e.g., pick from gallery)
        alternative = driver.find_element(AppiumBy.ID, "choose_from_gallery")
        assert alternative.is_displayed()
    
    def test_permission_rationale(self, driver):
        """
        Test that rationale is shown before second request (Android)
        """
        if driver.capabilities['platformName'] != 'Android':
            pytest.skip("Permission rationale is Android-specific behavior")
        
        # First denial
        driver.find_element(AppiumBy.ID, "access_contacts").click()
        driver.find_element(AppiumBy.ID, "com.android.permissioncontroller:id/permission_deny_button").click()
        
        # Try again immediately
        driver.find_element(AppiumBy.ID, "access_contacts").click()
        
        # Should show rationale dialog (custom UI explaining why needed)
        try:
            rationale = driver.find_element(AppiumBy.ID, "permission_rationale_dialog")
            assert rationale.is_displayed()
            
            # User can proceed to request again or cancel
            driver.find_element(AppiumBy.ID, "proceed_to_settings").click()
            
            # Opens settings app
            assert "Settings" in driver.find_element(AppiumBy.ID, "action_bar").text
            
        except NoSuchElementException:
            # On some Android versions, might just show system dialog again with "Don't ask again" option
            pass
    
    def test_dont_ask_again(self, driver):
        """
        Test 'Don't ask again' behavior
        """
        # Deny twice with don't ask again (Android)
        for _ in range(2):
            driver.find_element(AppiumBy.ID, "access_location").click()
            
            if driver.capabilities['platformName'] == 'Android':
                try:
                    checkbox = driver.find_element(AppiumBy.ID, "com.android.permissioncontroller:id/dont_ask_again")
                    checkbox.click()
                    driver.find_element(AppiumBy.ID, "permission_deny_button").click()
                except:
                    driver.find_element(AppiumBy.ID, "permission_deny_button").click()
        
        # Third attempt should not show dialog, immediately fail or show custom UI
        driver.find_element(AppiumBy.ID, "access_location").click()
        
        try:
            # Should not find system permission dialog
            dialog = driver.find_element(AppiumBy.ID, "com.android.permissioncontroller:id/permission_message")
            assert False, "Permission dialog shown after 'Don't ask again'"
        except:
            # Good - should show settings redirect instead
            settings_btn = driver.find_element(AppiumBy.ID, "open_settings")
            assert settings_btn.is_displayed()
    
    def test_permission_revocation_while_running(self, driver):
        """
        Test app behavior when permission revoked via Settings while app running
        """
        # Grant permission first
        self.test_camera_permission_grant(driver)
        
        # Background app
        driver.background_app(-1)
        
        # Revoke permission via ADB/settings
        if driver.capabilities['platformName'] == 'Android':
            driver.execute_script('mobile: shell', {
                'command': 'pm revoke com.example.app android.permission.CAMERA'
            })
        
        # Resume app
        driver.activate_app("com.example.app")
        
        # Try to use camera again
        driver.find_element(AppiumBy.ID, "take_another_photo").click()
        
        # Should handle gracefully (request again or show error)
        try:
            # Might crash if not handled - catch and verify
            error_handler = driver.find_element(AppiumBy.ID, "security_exception_handler")
            assert error_handler.is_displayed()
        except:
            # If no crash, verify permission request appears again
            pass
```

---

## **28.9 Memory and Storage**

Mobile devices have limited resources. Testing must verify graceful degradation when memory is low or storage is full.

### **28.9.1 Low Memory and Out of Memory (OOM)**

```python
class MemoryTesting:
    """
    Testing memory pressure scenarios
    """
    
    def test_low_memory_warning(self, driver):
        """
        Test handling of system low memory warnings
        """
        # Simulate memory pressure (requires root or specific APIs)
        if driver.capabilities['platformName'] == 'Android':
            # Fill memory to trigger low memory killer
            driver.execute_script('mobile: shell', {
                'command': 'am send-trim-memory com.example.app RUNNING_CRITICAL'
            })
        
        # App should release caches, save state
        status = driver.find_element(AppiumBy.ID, "memory_status")
        assert "Low memory" in status.text or "Clearing cache" in status.text
    
    def test_oom_recovery(self, driver):
        """
        Test app recovery after OOM kill
        """
        # Create significant state
        driver.find_element(AppiumBy.ID, "compose_message").click()
        driver.find_element(AppiumBy.ID, "message_body").send_keys("Important unsent message")
        
        # Background and kill app (simulate OOM)
        driver.background_app(-1)
        driver.execute_script('mobile: shell', {
            'command': 'am kill com.example.app'  # Kill background process
        })
        
        # Relaunch
        driver.activate_app("com.example.app")
        
        # Verify state restoration or graceful handling
        try:
            draft_restored = driver.find_element(AppiumBy.ID, "draft_message")
            assert "Important unsent message" in draft_restored.text
        except:
            # If not restored, should at least not crash and might show recovery message
            pass
    
    def test_large_image_handling(self, driver):
        """
        Test handling of large images without OOM
        """
        # Select very large image (10MB+)
        driver.find_element(AppiumBy.ID, "upload_photo").click()
        driver.find_element(AppiumBy.ID, "select_large_file").click()
        
        # Should show progress/compression rather than crash
        try:
            processing = driver.find_element(AppiumBy.ID, "compressing_image")
            assert processing.is_displayed()
            
            # Wait for completion or error (not crash)
            WebDriverWait(driver, 30).until(
                lambda d: d.find_element(AppiumBy.ID, "upload_complete").is_displayed() or
                         d.find_element(AppiumBy.ID, "file_too_large_error").is_displayed()
            )
        except WebDriverException:
            assert False, "App crashed on large image"
    
    def test_storage_full_scenario(self, driver):
        """
        Test behavior when device storage is full
        """
        # Fill storage (simulated or actual test device)
        if driver.capabilities['platformName'] == 'Android':
            # Create large file to fill space
            driver.execute_script('mobile: shell', {
                'command': 'dd if=/dev/zero of=/sdcard/fill.tmp bs=1M count=1000'
            })
        
        try:
            # Attempt operation requiring storage (download, save)
            driver.find_element(AppiumBy.ID, "download_file").click()
            
            # Should show storage full error, not crash
            error = driver.find_element(AppiumBy.ID, "insufficient_storage_error")
            assert error.is_displayed()
        finally:
            # Cleanup
            driver.execute_script('mobile: shell', {
                'command': 'rm /sdcard/fill.tmp'
            })
    
    def test_external_storage_unmount(self, driver):
        """
        Test handling of SD card removal (Android)
        """
        # Save file to external storage
        driver.find_element(AppiumBy.ID, "save_to_sd_card").click()
        
        # Simulate unmount (requires specific setup)
        # In real testing, physically remove SD card or use emulator controls
        
        # Try to access file
        driver.find_element(AppiumBy.ID, "view_saved_file").click()
        
        # Should handle gracefully
        error = driver.find_element(AppiumBy.ID, "storage_unavailable")
        assert error.is_displayed()
```

---

## **28.10 Camera and Media**

Camera integration requires testing various capture modes, permissions, storage handling, and media processing.

### **28.10.1 Camera Functionality Testing**

```python
class CameraTesting:
    """
    Testing camera integration
    """
    
    def test_photo_capture(self, driver):
        """
        Test taking a photo
        """
        driver.find_element(AppiumBy.ID, "camera_button").click()
        
        # Grant camera permission if needed
        self.handle_permission_if_present(driver, "camera")
        
        # Wait for camera preview
        preview = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((AppiumBy.ID, "camera_preview"))
        )
        assert preview.is_displayed()
        
        # Capture photo
        driver.find_element(AppiumBy.ID, "shutter_button").click()
        
        # Verify preview or saved state
        time.sleep(1)
        captured = driver.find_element(AppiumBy.ID, "captured_image_preview")
        assert captured.is_displayed()
        
        # Confirm save
        driver.find_element(AppiumBy.ID, "confirm_save").click()
        
        # Verify in gallery or list
        gallery = driver.find_element(AppiumBy.ID, "photo_gallery")
        photos = gallery.find_elements(AppiumBy.CLASS_NAME, "android.widget.ImageView")
        assert len(photos) > 0
    
    def test_camera_flash_modes(self, driver):
        """
        Test flash on/off/auto
        """
        driver.find_element(AppiumBy.ID, "camera_button").click()
        
        modes = ["flash_off", "flash_on", "flash_auto"]
        for mode in modes:
            driver.find_element(AppiumBy.ID, f"{mode}_button").click()
            
            # Take photo with each mode
            driver.find_element(AppiumBy.ID, "shutter_button").click()
            time.sleep(0.5)
            
            # Verify no crash
            assert driver.find_element(AppiumBy.ID, "captured_image_preview").is_displayed()
            driver.find_element(AppiumBy.ID, "retake_button").click()
    
    def test_front_rear_camera_switch(self, driver):
        """
        Test switching between front and rear cameras
        """
        driver.find_element(AppiumBy.ID, "camera_button").click()
        
        # Default is usually rear
        driver.find_element(AppiumBy.ID, "switch_camera").click()
        
        # Verify front camera active (may show face detection UI)
        front_indicator = driver.find_element(AppiumBy.ID, "front_camera_indicator")
        assert front_indicator.is_displayed()
        
        # Capture with front camera
        driver.find_element(AppiumBy.ID, "shutter_button").click()
        assert driver.find_element(AppiumBy.ID, "captured_image_preview").is_displayed()
    
    def test_photo_library_access(self, driver):
        """
        Test selecting from gallery/photos
        """
        driver.find_element(AppiumBy.ID, "choose_from_library").click()
        
        # Handle photo permission
        self.handle_permission_if_present(driver, "photos")
        
        # Select first photo
        if driver.capabilities['platformName'] == 'Android':
            # Android file picker
            driver.find_element(AppiumBy.XPATH, "//*[@text='Images']").click()
            driver.find_element(AppiumBy.CLASS_NAME, "android.widget.ImageView").click()
        else:
            # iOS Photo picker
            driver.find_element(AppiumBy.XPATH, "//XCUIElementTypeCell").click()
        
        # Verify selected
        selected_preview = driver.find_element(AppiumBy.ID, "selected_image_preview")
        assert selected_preview.is_displayed()
    
    def handle_permission_if_present(self, driver, type):
        """Helper to handle permission dialogs"""
        try:
            if driver.capabilities['platformName'] == 'Android':
                allow = driver.find_element(AppiumBy.ID, "com.android.permissioncontroller:id/permission_allow_button")
                allow.click()
            else:
                allow = driver.find_element(AppiumBy.XPATH, "//XCUIElementTypeButton[@name='OK']")
                allow.click()
        except:
            pass
    
    def test_video_recording(self, driver):
        """
        Test video capture
        """
        driver.find_element(AppiumBy.ID, "camera_button").click()
        driver.find_element(AppiumBy.ID, "video_mode").click()
        
        # Start recording
        driver.find_element(AppiumBy.ID, "record_button").click()
        time.sleep(3)  # Record 3 seconds
        
        # Stop recording
        driver.find_element(AppiumBy.ID, "stop_button").click()
        
        # Verify video preview
        video_preview = driver.find_element(AppiumBy.ID, "video_preview")
        assert video_preview.is_displayed()
        
        # Check duration
        duration = driver.find_element(AppiumBy.ID, "video_duration").text
        assert "00:03" in duration or "3" in duration

    def test_camera_low_light_simulation(self, driver):
        """
        Test camera in low light conditions (if hardware supports)
        """
        # This requires physical device testing or specific emulator features
        # Verify night mode toggle appears in low light
        pass
```

---

## **Chapter Summary**

### **Key Takeaways from Chapter 28:**

**Orientation and Display:**
- **Rotation Testing:** Verify data persistence and layout adaptation when switching between portrait and landscape; test rapid rotation stress
- **Foldable Devices:** Test folded (compact), unfolded (expanded), and half-folded (tabletop) postures; verify multi-pane layouts on expanded screens
- **Multi-Window:** Ensure app handles split-screen and picture-in-picture modes without crashes or data loss

**Background/Foreground:**
- **State Preservation:** Implement and test `onSaveInstanceState` / `SceneDelegate` to handle backgrounding, suspension, and system termination
- **Doze Mode:** Verify background jobs defer appropriately during device idle periods (Android)
- **App Switcher:** Ensure sensitive screens are obscured in recent apps (FLAG_SECURE) to prevent data leakage via screenshots

**Push Notifications:**
- **Delivery:** Test notification receipt in foreground (in-app handling), background (system tray), and terminated (cold start via tap) states
- **Rich Notifications:** Verify images, action buttons, and inline replies render correctly
- **Channels/Categories:** Ensure proper categorization for Android notification channels and iOS notification categories with appropriate sounds/badges
- **Permissions:** Test notification permission flows and graceful degradation when denied

**Location Services:**
- **Accuracy Levels:** Test high accuracy (GPS), battery saving (network), and device-only modes
- **Permissions:** Verify "While Using" vs "Always" permission handling; background location must have specific justification (iOS) or background location permission (Android 10+)
- **Geofencing:** Test boundary conditions (enter/exit events) with mocked locations
- **Mock Locations:** Validate app behavior with simulated GPS coordinates for testing location-based features without physical movement

**Biometric Authentication:**
- **Success/Failure:** Test successful fingerprint/Face ID match and fallback to passcode after failed attempts
- **Enrollment Changes:** Verify app detects new biometric enrollment and invalidates previous authentications (security requirement)
- **Availability:** Ensure graceful fallback to PIN/password when biometrics unavailable or not enrolled

**Deep Linking:**
- **Universal Links/App Links:** Test HTTPS URLs open app directly without disambiguation dialogs (requires domain verification)
- **Custom Schemes:** Verify `myapp://` URLs handle parameters correctly and navigate to appropriate screens
- **Deferred Deep Links:** Test attribution persistence through app install (Branch/Adjust/Firebase Dynamic Links)
- **Authentication:** Ensure deep links to protected resources redirect to login, then complete navigation post-authentication

**Permissions:**
- **Runtime Model:** Test permission request timing (should be contextual, not at launch)
- **Denial Handling:** Verify app functions gracefully when permissions denied (degraded features, not crashes)
- **Don't Ask Again:** Test permanent denial flow (should redirect to Settings rather than re-prompting)
- **Revocation:** Test behavior when permissions revoked via Settings while app is backgrounded

**Memory and Storage:**
- **OOM Handling:** Implement low memory callbacks to clear caches; test state restoration after OOM kill
- **Large Media:** Test image compression and chunked uploads to prevent memory exhaustion
- **Storage Full:** Verify graceful error handling when saving files with no space available
- **External Storage:** Test SD card unmount scenarios and media scanning behavior

**Camera and Media:**
- **Capture Modes:** Test photo/video, front/rear camera, and flash modes
- **Permissions:** Verify camera and microphone permission handling before capture
- **Storage:** Confirm captured media appears in gallery/photos and is accessible
- **Quality:** Test different resolution settings and compression ratios

---

## **📖 Next Chapter: Chapter 29 - Database Fundamentals for Testers**

Having mastered mobile-specific scenarios and interactions, **Chapter 29** transitions to data layer testing with **Database Fundamentals for Testers**.

In **Chapter 29**, you'll learn:

- **Database Architecture:** Understanding relational vs. NoSQL databases, tables, rows, columns, and relationships (1:1, 1:N, M:N)
- **SQL for Testers:** Essential SQL commands - SELECT, INSERT, UPDATE, DELETE, JOIN operations, and aggregate functions (COUNT, SUM, AVG)
- **Database Testing Types:** Schema testing, data integrity testing, stored procedure testing, trigger validation, and migration testing
- **CRUD Operations:** Create, Read, Update, Delete testing to ensure data persistence layer functions correctly
- **Data Integrity:** Entity integrity (primary keys), referential integrity (foreign keys), domain integrity (constraints), and user-defined integrity
- **Transaction Testing:** ACID properties (Atomicity, Consistency, Isolation, Durability), commit/rollback behavior, and deadlock detection
- **Test Data Management:** Strategies for creating valid test datasets, data anonymization for privacy compliance (GDPR), and data generation tools
- **Database Performance:** Index usage, query execution plans, slow query identification, and connection pool testing

**Chapter 29** provides the foundation for backend data validation, ensuring that the information flowing through your mobile applications is stored, retrieved, and maintained with integrity and performance.

**Continue to Chapter 29 to master database testing and complete your full-stack testing expertise!**